In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas


In [14]:
# import csvs
season_1617 = pd.read_csv("cleaned_bball_data/2016_17_nbaavgs.csv")
season_1718 = pd.read_csv("cleaned_bball_data/2017_18_nbaavgs.csv")
season_1819 = pd.read_csv("cleaned_bball_data/2018_19_nbaavgs.csv")
season_1920 = pd.read_csv("cleaned_bball_data/2019_20_nbaavgs.csv")
season_2021 = pd.read_csv("cleaned_bball_data/2020_21_nbaavgs.csv")
season_1920.columns

Index(['ID', 'Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'BPM', 'VORP'],
      dtype='object')

In [15]:
def year_cleaning(df, year):
    df['year'] = year
    df['u_player'] = df['Player'] + " " + df['year']
    df.drop()
    return df

In [16]:
season_1617 = year_cleaning(season_1617,"2017")
season_1617

,ID,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,...,STL,BLK,TOV,PF,PTS,PER,BPM,VORP,year,u_player
0,1,Álex Abrines,68,6,15.5,2.0,5.0,0.393,1.4,3.6,...,0.5,0.1,0.5,1.7,6.0,10.1,-1.6,0.1,2017,Álex Abrines 2017
1,2,Quincy Acy,38,1,14.7,1.8,4.5,0.412,1.0,2.4,...,0.4,0.4,0.6,1.8,5.8,11.8,-2.1,0.0,2017,Quincy Acy 2017
2,3,Steven Adams,80,80,29.9,4.7,8.2,0.571,0.0,0.0,...,1.1,1.0,1.8,2.4,11.3,16.5,-0.2,1.1,2017,Steven Adams 2017
3,4,Arron Afflalo,61,45,25.9,3.0,6.9,0.440,1.0,2.5,...,0.3,0.1,0.7,1.7,8.4,8.9,-3.6,-0.7,2017,Arron Afflalo 2017
4,5,Alexis Ajinça,39,15,15.0,2.3,4.6,0.500,0.0,0.1,...,0.5,0.6,0.8,2.0,5.3,12.9,-3.3,-0.2,2017,Alexis Ajinça 2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,482,Cody Zeller,62,58,27.8,4.1,7.1,0.571,0.0,0.0,...,1.0,0.9,1.0,3.0,10.3,16.7,0.2,1.0,2017,Cody Zeller 2017
482,483,Tyler Zeller,51,5,10.3,1.5,3.1,0.494,0.0,0.0,...,0.1,0.4,0.4,1.2,3.5,13.0,-2.4,0.0,2017,Tyler Zeller 2017
483,484,Stephen Zimmerman,19,0,5.7,0.5,1.6,0.323,0.0,0.0,...,0.1,0.3,0.2,0.9,1.2,7.3,-8.3,-0.2,2017,Stephen Zimmerman 2017
484,485,Paul Zipser,44,18,19.2,2.0,5.0,0.398,0.8,2.3,...,0.3,0.4,0.9,1.8,5.5,6.9,-4.4,-0.5,2017,Paul Zipser 2017


In [17]:
# merge df's into one df

data_frames = [season_1617, season_1718, season_1819, season_1920, season_2021]

seasons_df_merged = pd.concat(data_frames)
seasons_df_merged = seasons_df_merged.set_index('ID')
seasons_df_merged.head()

,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,STL,BLK,TOV,PF,PTS,PER,BPM,VORP,year,u_player
ID,,,,,,,,,,,,,,,,,,,,,
1,Álex Abrines,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,...,0.5,0.1,0.5,1.7,6.0,10.1,-1.6,0.1,2017,Álex Abrines 2017
2,Quincy Acy,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,...,0.4,0.4,0.6,1.8,5.8,11.8,-2.1,0.0,2017,Quincy Acy 2017
3,Steven Adams,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,...,1.1,1.0,1.8,2.4,11.3,16.5,-0.2,1.1,2017,Steven Adams 2017
4,Arron Afflalo,61,45,25.9,3.0,6.9,0.440,1.0,2.5,0.411,...,0.3,0.1,0.7,1.7,8.4,8.9,-3.6,-0.7,2017,Arron Afflalo 2017
5,Alexis Ajinça,39,15,15.0,2.3,4.6,0.500,0.0,0.1,0.000,...,0.5,0.6,0.8,2.0,5.3,12.9,-3.3,-0.2,2017,Alexis Ajinça 2017


In [18]:
# make "Player" its own df

player = [seasons_df_merged["u_player"]]
player_name = pd.concat(player, axis=1)
player_name.head()

,u_player
ID,
1,Álex Abrines 2017
2,Quincy Acy 2017
3,Steven Adams 2017
4,Arron Afflalo 2017
5,Alexis Ajinça 2017


In [5]:
# remove players from merged df
seasons_df_merged.drop('Player', inplace=True, axis=1)

seasons_df_merged.head()

,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
ID,,,,,,,,,,,,,,,,,,,,,
1,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,...,1.3,0.6,0.5,0.1,0.5,1.7,6.0,10.1,-1.6,0.1
2,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,...,3.0,0.5,0.4,0.4,0.6,1.8,5.8,11.8,-2.1,0.0
3,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,...,7.7,1.1,1.1,1.0,1.8,2.4,11.3,16.5,-0.2,1.1
4,61,45,25.9,3.0,6.9,0.440,1.0,2.5,0.411,2.0,...,2.0,1.3,0.3,0.1,0.7,1.7,8.4,8.9,-3.6,-0.7
5,39,15,15.0,2.3,4.6,0.500,0.0,0.1,0.000,2.3,...,4.5,0.3,0.5,0.6,0.8,2.0,5.3,12.9,-3.3,-0.2


In [6]:
# scale the merged df

nba_scaled = StandardScaler().fit_transform(seasons_df_merged)
print(nba_scaled[0:5])


[[ 0.84057258 -0.61076908 -0.439592   -0.52437437 -0.42458376 -0.45091985
   0.62442807  0.5117518   0.60858312 -0.92894692 -0.90533306 -0.53476297
   0.2484454  -0.56296384 -0.65147414  0.90934124 -0.69435493 -0.94452773
  -0.92358072 -0.72956628 -0.29148103 -0.7232388  -0.73439543  0.01954654
  -0.41575204 -0.40671383 -0.01074125 -0.36293347]
 [-0.36939837 -0.79921158 -0.526396   -0.6141206  -0.53204171 -0.27835016
   0.14212009 -0.05147736  0.80955228 -0.76379008 -0.69875794 -0.63684952
   0.16118468 -0.11909718 -0.10859817  0.20266437 -0.42813398 -0.13212948
  -0.23017264 -0.78580542 -0.53461345  0.01566955 -0.61067653  0.14750085
  -0.44802806 -0.16552948 -0.08451357 -0.44506989]
 [ 1.32456097  2.17817999  1.12288002  0.68719967  0.26314712  1.16578569
  -1.06364987 -1.17793568 -1.94372526  1.32819653  1.10139675  0.61174745
   0.59748829  0.47272504  0.85651465 -0.4610389   3.56518015  0.78858853
   1.68689678 -0.44837057  1.16731346  1.49348625  0.87395036  0.91522667
   0.43956

In [7]:
# Initialize PCA model
pca = PCA(n_components=3)

In [8]:
# Get 3 principal components for the iris data.
nba_pca = pca.fit_transform(nba_scaled)

In [9]:
# create PCA df
df_nba_pca = pd.DataFrame(
    data = nba_pca, columns = ["principal component 1", "principal component 2", "principal component 3"], index=seasons_df_merged.index)
df_nba_pca.head()

,principal component 1,principal component 2,principal component 3
ID,,,
1,-1.990251,-1.111335,1.675666
2,-1.747374,-0.165192,0.646957
3,3.790688,3.616649,-2.826681
4,-0.626987,-1.072793,0.961454
5,-1.312353,2.132255,-1.432294


In [10]:
pca.explained_variance_ratio_

array([0.5089292 , 0.13199011, 0.09148876])

In [11]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_nba_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [12]:
# Initialize the K-means model
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(df_nba_pca)

# Predict clusters
predictions = model.predict(df_nba_pca)

# Add the predicted class columns
player_name["class"] = model.labels_
player_name.head()


,Player,class
ID,,
1,Álex Abrines,4
2,Quincy Acy,4
3,Steven Adams,0
4,Arron Afflalo,4
5,Alexis Ajinça,4


In [13]:
# Player Search Bar

player_search = player_name[player_name["Player"] == ""]
player_search

,Player,class
ID,,


In [14]:
#sort by class

sorted_class = player_name.sort_values(["class"])
class_one = sorted_class[(sorted_class["class"] == 0)]
class_two = sorted_class[(sorted_class["class"] == 1)]
class_three = sorted_class[(sorted_class["class"] == 2)]
class_four = sorted_class[(sorted_class["class"] == 3)]
class_five = sorted_class[(sorted_class["class"] == 4)]



In [15]:
# class 1 df

df_one = pd.concat([class_one, seasons_df_merged], axis=1)
class_one_df = df_one.dropna()
class_one_df.tail(25)
#center_avg_vorp = class_one_df.mean()["BPM"]
#print(center_avg_vorp)

,Player,class,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
ID,,,,,,,,,,,,,,,,,,,,,
2419,JaVale McGee,0.0,46,2,14.7,3.1,6.1,0.511,0.1,0.5,...,5.2,0.8,0.4,1.2,1.3,2.6,7.3,17.0,-2.5,-0.1
2453,Nerlens Noel,0.0,64,41,24.2,2.1,3.5,0.614,0.0,0.0,...,6.4,0.7,1.1,2.2,1.0,2.8,5.1,14.8,1.4,1.3
2458,Jusuf Nurkić,0.0,37,37,23.8,4.6,9.0,0.514,0.3,0.8,...,9.0,3.4,1.0,1.1,2.0,2.8,11.5,20.5,2.5,1.0
2470,Cameron Oliver,0.0,4,0,21.8,4.8,8.3,0.576,1.0,3.3,...,5.3,1.3,0.5,1.0,0.8,2.8,10.8,17.2,-2.0,0.0
2488,Mason Plumlee,0.0,56,56,26.8,4.2,6.8,0.614,0.0,0.1,...,9.3,3.6,0.8,0.9,1.9,3.3,10.4,18.7,2.2,1.6
2489,Jakob Poeltl,0.0,69,51,26.7,3.8,6.2,0.616,0.0,0.0,...,7.9,1.9,0.7,1.8,1.2,2.5,8.6,17.5,0.8,1.3
2497,Bobby Portis,0.0,66,7,20.8,4.7,9.1,0.523,1.1,2.4,...,7.1,1.1,0.8,0.4,0.8,1.7,11.4,19.9,1.5,1.2
2499,Dwight Powell,0.0,58,19,16.7,2.1,3.3,0.619,0.1,0.4,...,4.0,1.1,0.6,0.5,0.7,2.3,5.9,16.8,0.6,0.6
2510,Naz Reid,0.0,70,15,19.2,4.3,8.3,0.523,0.9,2.5,...,4.6,1.0,0.5,1.1,1.0,2.6,11.2,18.2,-0.2,0.6


In [16]:
# class 2 df

df_two = pd.concat([class_two, seasons_df_merged], axis=1)
class_two_df = df_two.dropna()
class_two_df.tail(15)
#good_avg_vorp = class_two_df.mean()["BPM"]
#print(good_avg_vorp)

,Player,class,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
ID,,,,,,,,,,,,,,,,,,,,,
2526,Terry Rozier,1.0,69,69,34.5,7.4,16.4,0.450,3.2,8.3,...,4.4,4.2,1.3,0.4,1.9,1.7,20.4,17.1,1.6,2.2
2529,Domantas Sabonis,1.0,62,62,36.0,7.8,14.6,0.535,0.8,2.6,...,12.0,6.7,1.2,0.5,3.4,3.3,20.3,20.6,3.8,3.3
2537,Collin Sexton,1.0,60,60,35.3,8.8,18.4,0.475,1.6,4.4,...,3.1,4.4,1.0,0.2,2.8,2.6,24.3,18.0,-0.2,1.0
2540,Pascal Siakam,1.0,56,56,35.8,7.8,17.2,0.455,1.3,4.4,...,7.2,4.5,1.1,0.7,2.3,3.1,21.4,17.7,0.8,1.4
2542,Ben Simmons,1.0,58,58,32.4,5.6,10.1,0.557,0.1,0.2,...,7.2,6.9,1.6,0.6,3.0,2.9,14.3,18.3,2.9,2.3
2558,Jayson Tatum,1.0,64,64,35.8,9.5,20.6,0.459,2.9,7.6,...,7.4,4.3,1.2,0.5,2.7,1.9,26.4,21.3,3.9,3.4
2576,Karl-Anthony Towns,1.0,50,50,33.8,8.5,17.5,0.486,2.4,6.3,...,10.6,4.5,0.8,1.1,3.2,3.7,24.8,23.1,4.5,2.8
2584,Fred VanVleet,1.0,52,52,36.5,6.5,16.7,0.389,3.3,9.2,...,4.2,6.3,1.7,0.7,1.8,2.3,19.6,16.8,2.5,2.2
2589,Nikola Vučević,1.0,70,70,33.5,9.5,19.9,0.477,2.5,6.3,...,11.7,3.8,0.9,0.7,1.8,2.0,23.4,22.9,5.3,4.4


In [17]:
# class 3 df

df_three = pd.concat([class_three, seasons_df_merged], axis=1)
class_three_df = df_three.dropna()
class_three_df.tail(15)
#star_avg_vorp = class_three_df.mean()["BPM"]
#print(star_avg_vorp)

,Player,class,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
ID,,,,,,,,,,,,,,,,,,,,,
2487,Theo Pinson,2.0,17,0,2.0,0.1,0.5,0.111,0.0,0.5,...,0.3,0.1,0.0,0.0,0.1,0.2,0.1,-8.1,-15.7,-0.1
2490,Vincent Poirier,2.0,10,0,3.9,0.3,1.2,0.250,0.0,0.1,...,1.4,0.2,0.0,0.3,0.4,0.8,0.8,1.6,-14.8,-0.1
2511,Cameron Reynolds,2.0,5,0,8.0,1.2,3.6,0.333,0.6,2.4,...,1.0,0.4,0.0,0.0,0.0,0.4,3.0,6.5,-8.1,-0.1
2512,Nick Richards,2.0,18,0,3.5,0.2,0.5,0.444,0.0,0.1,...,0.6,0.1,0.0,0.0,0.2,0.4,0.8,5.5,-8.0,-0.1
2516,Andre Roberson,2.0,5,0,12.6,0.4,2.8,0.143,0.2,1.6,...,3.0,0.8,0.6,0.2,0.4,0.8,1.2,4.2,-6.1,-0.1
2520,Justin Robinson,2.0,9,0,9.8,0.8,2.3,0.333,0.4,1.6,...,0.8,1.0,0.3,0.0,0.2,1.1,2.3,6.0,-6.8,-0.1
2539,Iman Shumpert,2.0,2,0,5.5,0.5,2.0,0.250,0.0,1.5,...,0.5,0.0,0.5,0.0,0.5,0.0,1.0,-0.4,-12.6,0.0
2544,Deividas Sirvydis,2.0,20,0,6.7,0.7,2.0,0.350,0.5,1.4,...,1.5,0.3,0.1,0.0,0.3,0.5,2.1,5.7,-5.4,-0.1
2552,Cassius Stanley,2.0,24,0,3.9,0.5,1.8,0.302,0.1,0.5,...,0.8,0.0,0.0,0.1,0.0,0.6,1.5,5.5,-11.2,-0.2


In [18]:
# class 4 df

df_four = pd.concat([class_four, seasons_df_merged], axis=1)
class_four_df = df_four.dropna()
class_four_df.tail()

,Player,class,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
ID,,,,,,,,,,,,,,,,,,,,,
2604,Derrick White,3.0,36,32,29.6,5.2,12.6,0.411,2.3,6.8,...,3.0,3.5,0.7,1.0,1.3,2.5,15.4,15.0,1.3,0.9
2609,Kenrich Williams,3.0,66,13,21.6,3.3,6.2,0.533,0.8,1.8,...,4.1,2.3,0.8,0.3,1.2,2.2,8.0,14.0,-0.7,0.4
2610,Lou Williams,3.0,66,4,21.6,4.0,9.8,0.410,1.0,2.6,...,2.1,3.4,0.7,0.1,1.6,0.9,11.3,14.0,-2.5,-0.2
2611,Patrick Williams,3.0,71,71,27.9,3.6,7.4,0.483,0.8,1.9,...,4.6,1.4,0.9,0.6,1.4,1.8,9.2,10.5,-2.4,-0.2
2621,Delon Wright,3.0,63,39,27.7,3.8,8.2,0.463,1.0,2.7,...,4.3,4.4,1.6,0.5,1.3,1.2,10.2,16.3,2.2,1.8


In [19]:
# class 5 df

df_five = pd.concat([class_five, seasons_df_merged], axis=1)
class_five_df = df_five.dropna()
class_five_df.tail()

,Player,class,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
ID,,,,,,,,,,,,,,,,,,,,,
2614,D.J. Wilson,4.0,35,1,12.4,1.9,4.8,0.405,0.9,2.5,...,3.2,0.7,0.3,0.5,0.6,1.3,5.2,11.5,-3.3,-0.1
2615,Dylan Windler,4.0,31,0,16.5,1.8,4.2,0.438,0.8,2.5,...,3.5,1.1,0.6,0.4,1.0,1.2,5.2,10.8,-1.7,0.0
2616,Justise Winslow,4.0,26,1,19.5,2.8,8.1,0.352,0.4,2.1,...,4.5,1.9,0.6,0.5,1.4,1.8,6.8,6.2,-6.0,-0.5
2617,Cassius Winston,4.0,22,0,4.5,0.6,1.5,0.424,0.4,0.8,...,0.4,0.5,0.1,0.0,0.2,0.4,1.9,10.8,-3.2,0.0
2620,Robert Woodard II,4.0,13,0,3.5,0.6,1.5,0.400,0.1,0.5,...,1.2,0.2,0.0,0.2,0.1,0.3,1.5,12.5,-5.3,0.0
